In [ ]:
# pip freeze -r requirements.txt

In [9]:
import os
import re
import requests
import qrcode
from PIL import Image, ImageDraw, ImageFont
from dotenv import load_dotenv

# Load Spotify API credentials
load_dotenv()
CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")

In [ ]:
# Get Spotify token
def get_token():
    r = requests.post(
        "https://accounts.spotify.com/api/token",
        data={"grant_type": "client_credentials"},
        auth=(CLIENT_ID, CLIENT_SECRET),
    )
    return r.json()["access_token"]

# Search for a track
def get_track_info(artist, title, token):
    query = f"{title} {artist}"
    headers = {"Authorization": f"Bearer {token}"}
    params = {"q": query, "type": "track", "limit": 1}
    r = requests.get("https://api.spotify.com/v1/search", headers=headers, params=params)
    items = r.json().get("tracks", {}).get("items", [])
    if not items:
        return None
    track = items[0]
    url = track["external_urls"]["spotify"]
    release = track["album"].get("release_date", "????")
    year = release.split("-")[0]
    return url, year

# Make QR code
def make_qr(url, filename):
    img = qrcode.make(url)
    img.save(filename)

# Create graphic with title/artist/year
def make_text_card(title, artist, year, filename, width=600, height=300):
    img = Image.new("RGB", (width, height), "white")
    draw = ImageDraw.Draw(img)

    # Load a nicer font if available
    try:
        font = ImageFont.truetype("arial.ttf", 28)
    except:
        font = ImageFont.load_default()

    text = f"\"{title}\"\nby {artist}\n({year})"
    draw.multiline_text((30, 100), text, font=font, fill="black", spacing=10)
    img.save(filename)

# Parse songs.txt
def parse_songs(file_path):
    songs = []
    with open(file_path, "r") as f:
        for line in f:
            parts = re.split(r"\s{2,}", line.strip())
            if len(parts) == 2:
                artist, title = parts
                songs.append((artist.strip(), title.strip()))
    return songs

# ---- Main Execution ----
token = get_token()
songs = parse_songs("songs.txt")
os.makedirs("QR", exist_ok=True)
os.makedirs("cards", exist_ok=True)

for artist, title in songs:
    print(f"Processing: {title} by {artist}")
    info = get_track_info(artist, title, token)
    if not info:
        print("❌ Not found")
        continue
    url, year = info
    slug = f"{artist}_{title}".replace(" ", "_").replace("/", "_")

    qr_path = f"QR/{slug}.png"
    card_path = f"cards/{slug}_text.png"

    make_qr(url, qr_path)
    make_text_card(title, artist, year, card_path)

    print(f"✅ Done: {slug}")
    break

Processing: Paranoid by Black Sabbath
✅ Done: Black_Sabbath_Paranoid
Processing: Immigrant Song by Led Zeppelin
✅ Done: Led_Zeppelin_Immigrant_Song
Processing: Smoke on the Water by Deep Purple
✅ Done: Deep_Purple_Smoke_on_the_Water
Processing: Hell Bent for Leather by Judas Priest
✅ Done: Judas_Priest_Hell_Bent_for_Leather
Processing: Rock Bottom by UFO
✅ Done: UFO_Rock_Bottom
Processing: Highway to Hell by AC/DC
✅ Done: AC_DC_Highway_to_Hell
Processing: Overkill by Motorhead
✅ Done: Motorhead_Overkill
Processing: (Don't Fear) The Reaper by Blue Öyster Cult
✅ Done: Blue_Öyster_Cult_(Don't_Fear)_The_Reaper
Processing: Stargazer by Rainbow
✅ Done: Rainbow_Stargazer
Processing: In Trance by Scorpions
✅ Done: Scorpions_In_Trance
Processing: The Boys Are Back in Town by Thin Lizzy
✅ Done: Thin_Lizzy_The_Boys_Are_Back_in_Town
Processing: Detroit Rock City by KISS


KeyboardInterrupt: 